In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
import numpy as np

In [ ]:
from scraper.web_scraper import FinancialStatements, Attributes

In [ ]:
cik = '1786352'
form = '10-k'
def download_xml(cik, form):
    base = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type={}&dateb=&owner=include&count=100&search_text='.format(cik, form)
    heads = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

    content = requests.get(base, headers = heads)
    bs4_base = BeautifulSoup(content.text)
    years = 0

    path_dict = {}
    if not os.path.exists(cik):
        os.makedirs(cik)

    for hrefs in bs4_base.find_all('a', href=True):

        if hrefs['href'].split('.')[-1] == 'htm' and len(hrefs['href'].split('/')) == 7:
            if hrefs.parent.parent.td.get_text().lower() != '10-k':
                continue
            years += 1
            htm_url = 'https://www.sec.gov/' + hrefs['href']
            content = requests.get(htm_url, headers = heads)
            bs4_htm = BeautifulSoup(content.text)

            for htm_hrefs in bs4_htm.find_all('a', href=True):
                if htm_hrefs['href'].split('.')[-1] == 'xml':
                    x = htm_hrefs['href'].split('.')[-2].split('_')[-1]
                    if x == 'cal':
                        y = htm_hrefs['href'].split('/')[-1]
                        z = y.split('-')[1][:8]
                        path_dict[z] = {'cal':'', 'lab':'', 'htm':''}
                        break

            for htm_hrefs in bs4_htm.find_all('a', href=True):
                if htm_hrefs['href'].split('.')[-1] == 'xml':
                    if '_' not in htm_hrefs['href'].split('/')[-1]:
                        x = 'htm'
                    else:
                        x = htm_hrefs['href'].split('.')[-2].split('_')[-1]

                    if x == 'cal' or x == 'lab' or x == 'htm':
                        url = 'https://www.sec.gov/' + htm_hrefs['href']
                        bs4_end = BeautifulSoup(requests.get(url, headers = heads).text)
                        y = url.split('/')[-1]
                        
                        file_path = cik + '/' + y
                        path_dict[z][x] = file_path
                        f =  open(file_path, "w")
                        f.write(str(bs4_end))
                        f.close()
        if years >= 1:
            break

    return path_dict

try:
    path_dict = download_xml(cik, form)
except:
    print("bleh")

In [ ]:
path_dict

In [ ]:
df = pd.read_csv('base_comp_cik2.csv')
df = df[df['CIK'].isna() == False]

In [ ]:
all_paths = {}
fin_count = 0
attrib_count = 0
for i in df.index:
    cik = str(int(df.iloc[i]['CIK']))
    try:
        path_dict = download_xml(cik, form)
        all_paths[cik] = path_dict
        year = list(path_dict.keys())[0]
        calculation_file = path_dict[year]['cal']
        label_file = path_dict[year]['lab']
        extracted_file = path_dict[year]['htm']
        
        try:
            x = FinancialStatements(calculation_file, extracted_file)
            if x.financial_statements != {}:
                x.save_json('financial_statements/'+cik+'.json')
                fin_count+=1
        except:
            pass

        try:
            x = Attributes(label_file, extracted_file)
            if x.attributes != {}:
                x.save_json('attributes/'+cik+'.json')
                attrib_count+=1
        except:
            pass

    except:
        pass
    print(i)


In [ ]:
import json

In [ ]:
path_json = json.dumps(all_paths)

In [ ]:
with open('path_json', 'w') as f:
    f.write(path_json)